In [9]:
import os
os.chdir('../')
%pwd

'e:\\Chest_Cancer_Classification'

In [10]:
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf
import mlflow
import mlflow.keras
import dagshub
from urllib.parse import urlparse

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/deepakreddybora99/Chest_Cancer_Classification.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="deepakreddybora99" 
os.environ["MLFOW_TRACKING_PASSWORD"]= "c0101bb13c2695294099698c776a9bd9b8d3d443"

In [11]:
import tensorflow as tf

model=tf.keras.models.load_model("artifacts/training/model.h5")

In [12]:
import mlflow
from chestcancer.constants import *
from chestcancer.utils.common import read_yaml, create_directories, save_json

In [ ]:
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf
import mlflow
import mlflow.keras
import dagshub
from urllib.parse import urlparse

# Initialize DAGsHub MLflow Tracking
REPO_OWNER = "deepakreddybora99"
REPO_NAME = "Chest_Cancer_Classification"
dagshub.init(repo_owner=REPO_OWNER, repo_name=REPO_NAME, mlflow=True)

# Entity
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# Configuration
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        return EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/chest cancer dataset",
            mlflow_uri=f"https://dagshub.com/{repo_owner}/{repo_name}.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

# Component
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1./255, validation_split=0.30)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            
            # Register model if not using file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model", save_model_kwargs={"save_format": "h5"})

# Pipeline Execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-03-31 12:26:30,035: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as deepakreddybora99

[2025-03-31 12:26:30,035: INFO: helpers: Accessing as deepakreddybora99]
[2025-03-31 12:26:30,735: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/deepakreddybora99/Chest_Cancer_Classification "HTTP/1.1 200 OK"]
[2025-03-31 12:26:31,491: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "deepakreddybora99/Chest_Cancer_Classification"

[2025-03-31 12:26:31,491: INFO: helpers: Initialized MLflow to track repo "deepakreddybora99/Chest_Cancer_Classification"]


Repository deepakreddybora99/Chest_Cancer_Classification initialized!

[2025-03-31 12:26:31,491: INFO: helpers: Repository deepakreddybora99/Chest_Cancer_Classification initialized!]
[2025-03-31 12:26:31,508: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-31 12:26:31,511: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-31 12:26:31,513: INFO: common: created directory at: artifacts]
Found 95 images belonging to 2 classes.
6/6 [==============================] - 45s 7s/step - loss: 32.9053 - accuracy: 0.5053
[2025-03-31 12:27:17,867: INFO: common: json file saved at: scores.json]


2025/03/31 12:27:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-03-31 12:27:23,532: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmps34dyv6g\model\data\model\assets
[2025-03-31 12:27:24,598: INFO: builder_impl: Assets written to: C:\Users\hp\AppData\Local\Temp\tmps34dyv6g\model\data\model\assets]


c:\Users\hp\anaconda3\anacoda\envs\chest\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/03/31 12:28:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
